In [2]:
import yaml

with open('/code/scraped/omscs_courses.yaml') as f:
    courses = yaml.safe_load(f)

with open('/code/scraped/omscs_specializations.yaml') as f:
    specializations = yaml.safe_load(f)

with open('/code/context/resume.txt') as f:
    resume = f.read()

with open('/code/context/interests.txt') as f:
    interests = f.read()

In [3]:
# from source.omscs import format_courses, format_specializations

# print(format_specializations(specializations))

In [4]:
from source.omscs import format_courses, format_specializations


with open('/code/context/prompt.txt') as f:
    prompt = f.read()

prompt = prompt.\
    replace('{{resume}}', resume).\
    replace('{{interests}}', interests).\
    replace('{{specialization}}', format_specializations(specializations)).\
    replace('{{courses}}', format_courses(courses))
print(prompt)

Introduction: The user is interested in pursuing a master's degree and has provided a list of available courses with descriptions, specializations with core/required and elective courses, their resume in text form, and some personal context. The user seeks two types of recommendations for their study plan based on this information.

Objective 1: Provide a recommendation for a specialization and corresponding courses that best match the user's background, experience, and interests as outlined in their resume and personal context.

Objective 2: Provide a recommendation for a specialization and corresponding courses that address the user's knowledge gaps, as inferred from the provided information.

---

Here is the user's resume:

Title
====
Senior Data Scientist and ML Engineer

Overview
====
Data science and ML leader with 15+ years of experience spanning data science, machine learning, software development, and product/project management. Demonstrated ability to leverage data and stati

In [5]:
from llm_workflow.openai import num_tokens, MODEL_COST_PER_TOKEN

tokens = num_tokens(model_name='gpt-4-0125-preview', value=prompt)
cost = tokens * MODEL_COST_PER_TOKEN['gpt-4-0125-preview']['input']
print(tokens)
print(f'Cost: ${cost:.5f}')

21529
Cost: $0.21529


In [8]:
from llm_workflow.openai import OpenAIChat, OpenAIServerChat
from dotenv import load_dotenv
load_dotenv()

chat = OpenAIServerChat(
        base_url='http://host.docker.internal:1234/v1',
        # system_message=system_message,
        streaming_callback=lambda x: print(x.response, end='', flush=True),
        timeout=120,
    )

# chat = OpenAIChat(
#     model_name='gpt-4-0125-preview',
#     streaming_callback=lambda x: print(x.response, end='', flush=True),
# )
response = chat(prompt)
with open('/code/context/recommendations.txt', 'w') as f:
    f.write(response)

Hello! I don't have a name, but I'm here to help you with any questions or tasks you might have. How can I assist you today?

In [33]:
print(f"Total Cost:            ${chat.cost:.5f}")
print(f"Total Tokens:          {chat.total_tokens:,}")
print(f"Total Prompt Tokens:   {chat.input_tokens:,}")
print(f"Total Response Tokens: {chat.response_tokens:,}")

Total Cost:            $0.25270
Total Tokens:          22,788
Total Prompt Tokens:   21,547
Total Response Tokens: 1,241


---